# 04. Azure AI Document Intelligence - Custom Model

> https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/overview?view=doc-intel-4.0.0

## A. Create an AI Document Intelligence resource and set up environment to run notebook

**_Prerequsite_:** <br>

**AI Document Intelligence resource**: <br>
To create a AI Document Intelligence resource in your Azure subscription:
Please follow the steps as specified https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/create-document-intelligence-resource?view=doc-intel-4.0.0


Get your newly created Document Intelligence service in the Azure portal and on the **Keys and Endpoint** page, copy the **Key1** and **Endpoint** values and paste them in the code cell below, replacing **YOUR_FORM_KEY** and **YOUR_FORM_ENDPOINT**.

**Environment: **

1. **AML workspace**: Please ensure you have Python 3.10 version or above ie select **Python 3.10 - SDK v2** as kernel in AML Notebook. <br>
2. **VS Code**: Please select Python **Python 3.10** above or try to set up virtual envi by following steps https://code.visualstudio.com/docs/python/environments

Please skip this step if you are already done it

## B. Install AI Doc Intelligence library

In [1]:
# install azure-ai-formrecognizer python library and restart the kernal after installation
# ignore this step is you have already installed azure-ai-formrecognizer python library
%pip install azure-ai-formrecognizer --upgrade --user

Note: you may need to restart the kernel to use updated packages.


## C. Build custom model

- Please follow the steps to create custom model https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/how-to-guides/build-a-custom-model?view=doc-intel-4.0.0
- Sample docs are available to build custom model at https://dev.azure.com/datasciencesquad/ai-bootcamp-2024/_git/week2-search-formrecognizer-app-agents?path=/Day2/AI-Doc-Intelligence-Sample-Files <br>
- Upload the sample documents to train ie label the documents and use the one of the sample file to test the model <br>

## D. Setting up AI Doc Intelligence endpoint, key and custom model id

In [1]:
# Import the os module for interacting with the operating system
import os
# Import for handling resource not found errors
from azure.core.exceptions import ResourceNotFoundError
# Import for authenticating with the Azure service
from azure.core.credentials import AzureKeyCredential
# Import for analysing forms
from azure.ai.formrecognizer import DocumentAnalysisClient, AnalyzeResult

# set `<your-endpoint>` and `<your-key>` variables with the values from the Azure portal
END_POINT = "XXXX"
END_POINT_KEY = "XXXX"
# custom model id that you have created
CUSTOM_MODEL_ID = "XXXX"

# create form recognizer client
form_recognizer_client = DocumentAnalysisClient(END_POINT, AzureKeyCredential(END_POINT_KEY))

## E. Extract the document insights using custom model

In [3]:
# sample document
# You can change the URL pointing to your sample custom docs but ensure you provide appropriate access
# sample documents available in the following page https://dev.azure.com/datasciencesquad/ai-bootcamp-2024/_git/week2-search-formrecognizer-app-agents?path=/Day2/AI-Doc-Intelligence-Sample-Files to train and create custom model
# make sure you enable at least read only access for the doc from your storage container account
sampleUrl = "XXX"

poller = form_recognizer_client.begin_analyze_document_from_url(CUSTOM_MODEL_ID,sampleUrl)
print(poller)

result = poller.result()

print(result.documents)

for idx, document in enumerate(result.documents):
    print(f"--------Analyzing document #{idx + 1}--------")
    print(f"Document has type {document.doc_type}")
    print(f"Document has document type confidence {document.confidence}")
    print(f"Document was analyzed with model with ID {result.model_id}")
    for name, field in document.fields.items():
        field_value = field.value if field.value else field.content
        print(
            f"......found field of type '{field.value_type}' with value '{field_value}' and with confidence {field.confidence}"
        )

# iterate over tables, lines, and selection marks on each page
for page in result.pages:
    print(f"\nLines found on page {page.page_number}")
    for line in page.lines:
        print(f"...Line '{line.content}'")
    for word in page.words:
        print(f"...Word '{word.content}' has a confidence of {word.confidence}")
    if page.selection_marks:
        print(f"\nSelection marks found on page {page.page_number}")
        for selection_mark in page.selection_marks:
            print(
                f"...Selection mark is '{selection_mark.state}' and has a confidence of {selection_mark.confidence}"
            )

for i, table in enumerate(result.tables):
    print(f"\nTable {i + 1} can be found on page:")
    for region in table.bounding_regions:
        print(f"...{region.page_number}")
    for cell in table.cells:
        print(
            f"...Cell[{cell.row_index}][{cell.column_index}] has text '{cell.content}'"
        )

[AnalyzedDocument(doc_type=customreceiptlrl, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=0.0, y=0.0), Point(x=8.2639, y=0.0), Point(x=8.2639, y=11.6806), Point(x=0.0, y=11.6806)]), BoundingRegion(page_number=2, polygon=[Point(x=0.0, y=0.0), Point(x=8.2639, y=0.0), Point(x=8.2639, y=11.6806), Point(x=0.0, y=11.6806)]), BoundingRegion(page_number=3, polygon=[Point(x=0.0, y=0.0), Point(x=8.2639, y=0.0), Point(x=8.2639, y=11.6806), Point(x=0.0, y=11.6806)]), BoundingRegion(page_number=4, polygon=[Point(x=0.0, y=0.0), Point(x=8.2639, y=0.0), Point(x=8.2639, y=11.6806), Point(x=0.0, y=11.6806)]), BoundingRegion(page_number=5, polygon=[Point(x=0.0, y=0.0), Point(x=8.2639, y=0.0), Point(x=8.2639, y=11.6806), Point(x=0.0, y=11.6806)]), BoundingRegion(page_number=6, polygon=[Point(x=0.0, y=0.0), Point(x=8.2639, y=0.0), Point(x=8.2639, y=11.6806), Point(x=0.0, y=11.6806)]), BoundingRegion(page_number=7, polygon=[Point(x=0.0, y=0.0), Point(x=8.2639, y=0.0), Point(x=8.2639, y=1

## F. Extracted document insights/ response as table of Key Value Pair using custom model

In [4]:
# Get the document insights as key / value table
from tabulate import tabulate

data = []

# Display key value pairs
for idx, document in enumerate(result.documents):
    print()
    print("--------Analyzing document #{}--------".format(idx + 1))
    print("Document has type {}".format(document.doc_type))
    print("Document has document type confidence {}".format(
        document.confidence))
    print("Document was analyzed with model with ID {}".format(
        result.model_id))
    print()
    for name, field in document.fields.items():
        field_value = field.value if field.value else field.content
        if field.value_type != 'list':
            data.append([name, field.value, field.confidence])

data.sort()
print(tabulate(data, headers=[
    'Label', 'Value', 'Confidence'], tablefmt='fancy_grid'))

# Display table data
for i, table in enumerate(result.tables):

    row_index = 1
    hdr = []
    rows = []
    row = []

    print("\nTable {} can be found on page:".format(i + 1))
    # for region in table.bounding_regions:
    #     print("...{}".format(i + 1, region.page_number))

    for cell in table.cells:
        if cell.row_index == 0:
            hdr.append(cell.content)
        else:
            if row_index != cell.row_index:
                rows.append(row)
                row_index = cell.row_index
                row = []

            row.append(cell.content)

    rows.append(row)
    print(tabulate(rows, headers=hdr, tablefmt='fancy_grid'))


--------Analyzing document #1--------
Document has type customreceiptlrl
Document has document type confidence 0.001
Document was analyzed with model with ID customreceiptlrl

╒══════════════════════════╤═════════╤══════════════╕
│ Label                    │ Value   │   Confidence │
╞══════════════════════════╪═════════╪══════════════╡
│ CustomerAddress          │         │        0.79  │
├──────────────────────────┼─────────┼──────────────┤
│ CustomerAddressRecipient │         │        0.792 │
├──────────────────────────┼─────────┼──────────────┤
│ CustomerName             │         │        0.711 │
├──────────────────────────┼─────────┼──────────────┤
│ InvoiceDate              │         │        0.247 │
├──────────────────────────┼─────────┼──────────────┤
│ MerchantAddress          │         │        0.494 │
├──────────────────────────┼─────────┼──────────────┤
│ MerchantName             │         │        0.423 │
├──────────────────────────┼─────────┼──────────────┤
│ MerchantPho

## G. Extracted document insights/ response as a JSON format

In [5]:
# Import necessary libraries
# json for handling JSON data
# datetime and time for generating timestamps
# AzureJSONEncoder for serializing Python objects to JSON
# urlparse for parsing URLs
import json
import datetime
import time
from azure.core.serialization import AzureJSONEncoder
from urllib.parse import urlparse


# generate the unique file name
filename = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d%H%M%S')+"_"+os.path.splitext(os.path.basename(urlparse(sampleUrl).path))[0]

# parse and format the model response json 
# convert the received model to a dictionary
analyze_result_dict = result.to_dict()

# save the dictionary as JSON content in a JSON file, use the AzureJSONEncoder
# to help make types, such as dates, JSON serializable
with open(str(filename), 'w') as f:
        json.dump(analyze_result_dict, f, cls=AzureJSONEncoder,indent=4)

# convert the dictionary back to the original model
model = AnalyzeResult.from_dict(analyze_result_dict)
print("--------------JSON Response from Model Starts---------------------")
# use the model as normal
print("Model ID: '{}'".format(model.model_id))
print("Number of pages analyzed {}".format(len(model.pages)))
print("API version used: {}".format(model.api_version))
print(json.dumps(analyze_result_dict,cls=AzureJSONEncoder,indent=4))
print("--------------JSON Response from Model Ends---------------------")

--------------JSON Response from Model Starts---------------------
Model ID: 'customreceiptlrl'
Number of pages analyzed 10
API version used: 2023-07-31


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

